In [ ]:
!pip install --upgrade setuptools
!pip install --upgrade gradio
!pip install ipywidgets
!pip install openai
!pip install pandas
!pip install tabulate
!pip install python-dotenv

In [ ]:
!pip install -e ../

In [1]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from dotenv import dotenv_values
from typechat import Failure, TypeChatTranslator, TypeChatValidator, create_language_model
from examples.calendar import schema as calendar

In [3]:
model = create_language_model(dotenv_values())
validator = TypeChatValidator(calendar.CalendarActions)
translator = TypeChatTranslator(model, validator, calendar.CalendarActions)

In [10]:
import pandas

async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        result = result.value
        df = pandas.DataFrame.from_dict(result["actions"])
        return f"Translation Succeeded! ✅\n Table View \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n"

def get_examples():
    example_prompts = []
    with open('../../examples/calendar/src/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts

In [11]:
import gradio as gr

gr.ChatInterface(get_translation, title="📅 Calendar", examples=get_examples()).launch()


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
